In [ ]:
!pip install python-binance
!pip install yfinance
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy 
import sklearn as sk
from binance import ThreadedWebsockManger
from binance.client import Client
from datetime import datetime, timedelta

General account/system Info

In [ ]:
api_key = "B5c6OfFGReHGporT9cjPJ"
secret_key = "KHnfhemKkNNDJnekek"

client = Client(api_key = api_key, api_secret = secret_key, tld = "us")
print(client)

client.get_account()
client.ping()
client.get_system_status()

account = client.get_account()
print(account)
print(account["accountType"])

pd.to_datatime(account["updateTime"], unit = "ms")
account["balances"]

df = pd.DataFrame(account["balances"])
print(df, df.info())

df.free = pd.to_numeric(df.free, errors = "coerce")
df.locked = pd.to_numeric(df.locked, errors = "coerce")
print(
  df.loc[df.free > 0],
  client.get_asset_balance(asset = "BTC"),
  client.get_asset_balance(asset = "ETH"),
  float(client.get_asset_balance(asset = "ETH")["Free"])
)

snap = client.get_account_snapshot(type = "SPOT")
snap = pd.json_normalize(snap["snapshotVos"])
print(snap)

snap["data.balances"][0]
client.get_exchange_info()["rateLimits"]
client.get_all_coins_info()

coins = pd.DataFrame(client.get_all_coins_info())
print(coins)

coins.loc[coins.coin == "BTC"]
coins.loc[coins.coin == "EUR"]

client.get_trade_fee(symbol = "BTCUSDT")
client.get_trade_fee(symbol = "BTCEUR")

Getting Market Data 

In [ ]:
client.get_symbol_ticker(symbol = "BTCUSDT")
float(client.get_symbol_ticker(symbol = "BTCEUR")["prices"])
client.get_avg_price(symbol = "BTCUSDT")

prices = client.get_all_tickers()
print(prices)

df = pd.DataFrame(prices)
print(df)

print(
  df[df.symbol.str.contains("ETH")],
  df[df.symbol.str.contains("BTC") & df.symbol.str.contains("USD")],
  df[df.symbol.str.contains("BTC") & df.symbol.str.contains("EUR")]
)

last24 = client.get_ticker(symbol = "BTCUSDT")

print(
    last24,
    last24["openTime"],
    pd.to_datetime(last24["openTime"], unit = "ms"),
    pd.to_datetime(last24["closeTime"], unit = "ms")
)

open_price = float(last24["closePrice"])
high_price = float(last24["highPrice"])
low_price = float(last24["lowPrice"])
close_price = float(last24["lastPrice"])

print(open_price, high_price, low_price)
print(close_price - open_price)
print((close_price/open_price - 1) * 100)

Getting Historical Data(OHLC & Volume)

In [ ]:
timestamp = client._get_earliest_vaild_timestamp(symbol = "BTCUSDT", intreval = "id")
print(timestamp)
print(pd.to_datetime(timestamp, unit = "ms"))

bars = client.get_historical_kliness(symbol = "BTCUSDT", intreval = "1d", start_str = timestamp, limit = 1000)
print(bars)

df = pd.DataFrame(bars)
print(df)

df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
print(df)

df.columns = ["open time", "open", "high", "low", "close", 
              "volume", "clos time",  "quote asset volume",
              "number of trades", "taker buy base asset volume",
              "ignore", "date"]

df = df[["Date", "Open", "high", "low", "close", "volume"]].copy()
print(df)

df.set_index("Date", inplace = True)
print(df, df.info())

for column in df.columns:
  df[column] = pd.to_numeric(df[column], errors = "coerce")

def get_history(symbol, intreval, start, end = None):
  bars = client.get_historical_kliness(symbol = symbol, intreval = intreval, start_str = start, end_str = end ,limit = 1000)

  df = pd.DataFrame(bars)
  df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
  df.columns = ["open time", "open", "high", "low", "close", 
              "volume", "clos time",  "quote asset volume",
              "number of trades", "taker buy base asset volume",
              "ignore", "date"]
  df = df[["Date", "Open", "high", "low", "close", "volume"]].copy()
  df.set_index("Date", inplace = True)
  for column in df.columns:
    df[column] = pd.to_numeric(df[column], errors = "coerce")

  return df

print(timestamp)

df = get_history(symbol = "BTCUSDT", intreval = "1d", start = timestamp)
print(df, df.info())

df = get_history(symbol = "BTCUSDT", intreval = "1d", start = "2021-01-04", end = "2021-06-30") # per day
print(df)

df = get_history(symbol = "BTCUSDT", intreval = "1w", start = "2021-01-01", end = "2021-06-30") # per week
print(df)

df = get_history(symbol = "BTCUSDT", intreval = "1m", start = "2021-01-04", end = "2021-06-30") # per month
print(df)

df = get_history(symbol = "BTCUSDT", intreval = "1h", start = "2021-01-04 10:00:00", end = "2021-06-30 16:00:00") # per hour
print(df)

now = datetime.utcnow()
print(now)

two_hours_before = now - timedelta(hours = 2)
print(two_hours_before)

print(str(two_hours_before))

df = get_history(symbol = "BTCUSDT", intreval = "1m", start = str(two_hours_before))
print(df)

Streaming Market Data 

In [ ]:
def stream_data(msg):
  print(msg)

twn = ThreadedWebsockManger(tld = "us")
twn.start()
print(twn)

twn.start_symbol_miniticker_socket(callback = stream_data, symbol = "BTCUSDT")
twn.stop()

Collecting real-time Candles

In [ ]:
df = pd.DataFrame(columns = ["Open", "High", "Low", "Close", "volume", "Complete"])
print(df)

def stream_candles(msg):
  event_time = pd.to_datetime(msg["E"], unit = "ms")
  start_time = pd.to_datetime(msg["K"]["t"], unit = "ms")
  first = float(msg["k"]["o"])
  high = float(msg["k"]["h"])
  low = float(msg["k"]["l"])
  close = float["k"]["c"]
  volume = float(msg["k"]["v"])
  complete = msg["k"]["x"]
  print("Time: {} | Price: {}".format(event_time, close))
  df.loc[start_time] = [first, high, low, close, volume, complete]

twn = ThreadedWebsocketManager(tld = "us")
twn.start()

twn.start_kline_socket(callback = stream_candles, symbol = "BTCUSDT", intreval = "1m")
twn.stop()


Creating a Test order

In [ ]:
client = Client(api_key, api_secret = secret_key, tld = "com")
order = client.create_test_order(symbol = "BTCEUR", side = "BUY", type = "MARKET", quantity = 0.1)
print(order)

client.get_account()

Placing Limit Orders

In [ ]:
# Buy Limit
client.get_symbol_ticker(symbol = "BTCUSDT")

order1 = client.create_order(symbol = "BTCUSDT", side = "BUY", type = "Limit", quantity = 0.1, timeInForce = "GTC", price = 50000)
print(order1)

order_id_buy = order1["orderId"]
print(order_id_buy)

client.get_order(symbol = "BTCUSDT", orderId = order_id_buy)

# Sell Limit
order2 = client.create_order(symbol = "BTCUSDT", side = "SELL", type = "Limit", quantity = 0.1, timeInForce = "GTC", price = 50000)
print(order2)

order_id_sell = order2["orderId"]
print(order_id_sell)

open_orders = client.get_open_orders(symbol = "BTCUSDT")
print(open_orders)

print(pd.DataFrame(open_orders))

Bresult = client.cancel_order(symbol = "BTCUSDT", orderId = order_id_buy)
print(Bresult)

Sresult = client.cancel_order(symbol = "BTCUSDT", orderId = order_id_sell)
print(Sresult)

Data Preparation

In [ ]:
data = pd.read_csv('bitcoin.csv', parse_date = ["Date"], index_col = "Date")
data['vol_ch'] = np.log(data.volume.div(data.Volume.shift(1)))
print(data)

data.vol_ch.plot(kind = "hist", bins = 100, figsize = (12,8))
plt.show()

data.vol_ch.nsmallest(20)
data.vol_ch.nlargest(20)

data.loc[data.vol_ch > 3, 'vol_ch'] = np.nan
data.loc[data.vol_ch < -3, "vol_ch"] = np.nan

print(data.info())

Explanatory Data Analysis: Finanical Returns and Trading Volume

In [ ]:
plt.scatter(x = data.vol_ch, y = data.returns)
plt.xlabel("Volume Change")
plt.ylabel("Returns")
plt.show()

pd.qcut(data.returns, q = 10)
data["ret_cat"] = pd.qcut(data.returns, q = 10, labels = [-5,-4,-3,-2,-1,1,2,3,4,5])
print(data)

data.ret_cat.value_counts()
data["vol_cat"] = pd.qcut(data.vol_ch, g = 10, labels = [-5,-4,-3,-2,-1,1,2,3,4,5])

matrix = pd.crosstab(data.vol_cat, data.ret_cat)
print(matrix)

plt.figure(figsize = (12,8))
sns.set(font_scale = 1)
sns.heatmap(matrix, cmap = "RdY1bu_r", annot = True, robust = True, fmt = ".0f")
plt.show()

data.vol_cat.shfit()
matrix = pd.crosstab(data.vol_cat.shift(), data.ret_cat.shift(),
                     values = data.returns, afffunc = np.mean)
print(matrix)

plt.figure(figsize = (15, 12))
sns.set(font_scale = 1)
sns.heatmap(matrix, cmap = "RdY1bu_r", annot = True, robust = True, fmt = ".5f")
plt.show()



Fromulating a Long-only Price/Volume Trading Strategy

In [ ]:
data["position"] = 1
print(data)

return_thresh = np.percentlie(data.returns.dropna(), 90)
print(return_thresh)

condl = data.returns >= return_thresh
print(condl)

volume_thresh = np.percentlie(data.vol_ch.dropna(), [5,20])
print(volume_thresh)

cond2 = data.vol_ch.between(volume_thresh[0], volume_thresh[1])
print(cond2)

data.loc[condl & cond2, 'position'] = 0
print(data)

data.position.value_counts()
print(data)

data.loc[:, 'positon'].plot(figszie = (12, 8))
plt.show()

data.loc["2019-06", "postion"].plot(figszie = (12,8))
plt.show()